In [1]:
import json

with open('./config.json', 'r') as f:
    config = json.load(f)

In [2]:
service_key = config['scraping']['serviceKey']
TEMPLATE = "http://apis.data.go.kr/B551011/KorService1/areaBasedList1?areaCode=%d&numOfRows=10000000&pageNo=1&MobileOS=ETC&MobileApp=AppTest&_type=json&listYN=Y&arrange=A"

In [3]:
import os
path = '../지역코드'

area_code_file_names = list(filter(lambda fn: fn.endswith('.json'), os.listdir(path)))
print('지역 코드 파일 목록:', area_code_file_names)

area_codes = list(map(lambda fn: (int(fn.split('_')[0]), fn.split('_')[1].split('.')[0]), area_code_file_names))
print('지역 코드별 지역명:', area_codes)

지역 코드 파일 목록: ['01_서울.json', '02_인천.json', '03_대전.json', '04_대구.json', '05_광주.json', '06_부산.json', '07_울산.json', '08_세종.json', '31_경기.json', '32_강원.json', '33_충북.json', '34_충남.json', '35_경북.json', '36_경남.json', '37_전북.json', '38_전남.json', '39_제주.json']
지역 코드별 지역명: [(1, '서울'), (2, '인천'), (3, '대전'), (4, '대구'), (5, '광주'), (6, '부산'), (7, '울산'), (8, '세종'), (31, '경기'), (32, '강원'), (33, '충북'), (34, '충남'), (35, '경북'), (36, '경남'), (37, '전북'), (38, '전남'), (39, '제주')]


In [4]:
"""
데이터베이스 드랍 + 테이블 생성
"""

import pymysql

# 0. 설정 읽기
database_connection_config = config['database']

# 1. mysql 연결
conn = pymysql.connect(**database_connection_config)

# 2. 커서 생성
cursor = conn.cursor()

# 3. 테이블 드랍
cursor.execute("DROP TABLE IF EXISTS attractions;")

# 3. 테이블 재생성
cursor.execute("""
CREATE TABLE attractions (
    id              INT PRIMARY KEY NOT NULL,
    addr1           VARCHAR(100)    NOT NULL DEFAULT '',
    addr2           VARCHAR(100)    NOT NULL DEFAULT '',
    area_code       INT             NOT NULL,
    category_1      CHAR(3),
    category_2      CHAR(5),
    category_3      CHAR(9),
    content_type_id INT             NOT NULL,
    first_image     VARCHAR(200)    NOT NULL DEFAULT '',
    second_image    VARCHAR(200)    NOT NULL DEFAULT '',
    longitude       FLOAT           NOT NULL,
    latitude        FLOAT           NOT NULL,
    sigungucode     CHAR(3)         NOT NULL DEFAULT '',
    tel             VARCHAR(100),
    title           VARCHAR(100)
);
""")

# 데이터 저장 및 연결 끊기
conn.commit()
conn.close()

In [5]:
from time import sleep
import json

import requests
TIME_WAIT_SEC = 3

BASE_SQL = """
INSERT INTO attractions(addr1, addr2, area_code, category_1, category_2, category_3, id, 
                        content_type_id, first_image, second_image, longitude, latitude,
                        sigungucode, tel, title)
VALUES """


def clean_input(string):
    return string.replace("'", '"')

failed_areas = []
total_num_of_rows = 0
for success_cnt, (area_code, name) in enumerate(area_codes, 1):
    value_segments = []
    url = (TEMPLATE % area_code) + '&serviceKey=' + service_key
    print('\n지역:', name)
    print('url:', url)

    response: requests.Response = requests.get(url)
    raw = response.content.decode('utf8').replace("'", '"').replace("`", '')
    body = json.loads(raw)['response']['body']
    num_of_rows = body['numOfRows']
    total_num_of_rows += num_of_rows

    # body parsing
    keys = ['addr1', 'addr2', 'areacode', 'cat1', 'cat2', 'cat3', 'contentid', 'contenttypeid', 'firstimage']
    for document in body['items']['item']:
        segment = f"""('{clean_input(document["addr1"])}', '{clean_input(document["addr2"])}', '{clean_input(document["areacode"])}', '{clean_input(document["cat1"])}', 
                     '{clean_input(document["cat2"])}', '{clean_input(document["cat3"])}', {clean_input(document["contentid"])}, {clean_input(document["contenttypeid"])}, 
                     '{clean_input(document["firstimage"])}', '{clean_input(document["firstimage2"])}', {clean_input(document["mapx"])}, {clean_input(document["mapy"])}, 
                     '{clean_input(document["sigungucode"])}', '{clean_input(document["tel"])}', '{clean_input(document["title"])}') """
        value_segments.append(segment)

    print('>>>', num_of_rows, f'건 ({round(success_cnt / len(area_codes) * 100, 1)} %)')
    total_insert_sql = BASE_SQL + ', '.join(value_segments) + ';'
    with open(f'./{name}_insert.sql', 'w', encoding='utf-8') as f:
        f.write(total_insert_sql)

    conn = pymysql.connect(**database_connection_config)
    cursor = conn.cursor()
    try:
        cursor.execute(total_insert_sql)
        conn.commit()
        print('done')
    except Exception as e:
        print(e)
        failed_areas.append(name)
        conn.rollback()
    finally:
        conn.close()

    sleep(TIME_WAIT_SEC)

print('완료 : 총', total_num_of_rows, '건')


지역: 서울
url: http://apis.data.go.kr/B551011/KorService1/areaBasedList1?areaCode=1&numOfRows=10000000&pageNo=1&MobileOS=ETC&MobileApp=AppTest&_type=json&listYN=Y&arrange=A&serviceKey=rf2THTSq3Rnvz7yUJWlRW2YjsfVvxufJssCC4SgBAcoGXoIE%2FfZH72XMDiy4AYxbmYQXRGYxAFKO1%2BAKnL91rg%3D%3D
>>> 7501 건 (5.9 %)
done

지역: 인천
url: http://apis.data.go.kr/B551011/KorService1/areaBasedList1?areaCode=2&numOfRows=10000000&pageNo=1&MobileOS=ETC&MobileApp=AppTest&_type=json&listYN=Y&arrange=A&serviceKey=rf2THTSq3Rnvz7yUJWlRW2YjsfVvxufJssCC4SgBAcoGXoIE%2FfZH72XMDiy4AYxbmYQXRGYxAFKO1%2BAKnL91rg%3D%3D
>>> 1974 건 (11.8 %)
done

지역: 대전
url: http://apis.data.go.kr/B551011/KorService1/areaBasedList1?areaCode=3&numOfRows=10000000&pageNo=1&MobileOS=ETC&MobileApp=AppTest&_type=json&listYN=Y&arrange=A&serviceKey=rf2THTSq3Rnvz7yUJWlRW2YjsfVvxufJssCC4SgBAcoGXoIE%2FfZH72XMDiy4AYxbmYQXRGYxAFKO1%2BAKnL91rg%3D%3D
>>> 781 건 (17.6 %)
done

지역: 대구
url: http://apis.data.go.kr/B551011/KorService1/areaBasedList1?areaCode=4&numOfRow

In [6]:
print('failed:', failed_areas)

failed: ['경기', '경남']
